# TO DO LIST:
 1. Make file opener robust enough to open multiple midi files and once and upload them to memory for training purposes only. The final version will work with only one file being read (access midi data)  
 2. Make MIDI file conversion robust enough to work with multiple files. Again for training only, not needed for final version on hardware
 3. snipNoteSequence() needs more robustness. Edit function to allow for both a beginning location and count so that you can select a portion of a song. The best version would allow for time_stamps, if we have time (Train Dataset)
 4. function generation must return the new outputs so that it can be sent to other hardware (Train Dataset) **IMPORTANT**
 5. Decide "num_steps" correct sizing when generating a new chord. Seems their is a buffer of unkown length that we must figure out. **Possible Bug Creator**

# Chord Progession Model
This Collab Serves as the training notebook for our Chord Progression Model. The following model is built on the magenta "Melody RNN Model". The model was retrained using MIDI Chord Progression Files.  

*This model also serves as the documentation for the model itself. Including how to implement for your own research.*

# IMPORTS

In [ ]:
#IMPORT ALL MODEL DEPENDENCIES
#@test {"output": "ignore"}
print('Installing dependencies...')
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi

!pip install -qU magenta

# Hack to allow python to pick up the newly-installed fluidsynth lib. 
# This is only needed for the hosted Colab environment.
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library

print('Importing libraries and defining some helper functions...')
from google.colab import files

import magenta
import note_seq
import tensorflow
#originally on twinkle twinkle magenta cell now importing here
from note_seq.protobuf import music_pb2

print('🎉 Done!')
print(magenta.__version__)
print(tensorflow.__version__)

Installing dependencies...
Selecting previously unselected package fluid-soundfont-gm.
(Reading database ... 146442 files and directories currently installed.)
Preparing to unpack .../fluid-soundfont-gm_3.1-5.1_all.deb ...
Unpacking fluid-soundfont-gm (3.1-5.1) ...
Selecting previously unselected package libfluidsynth1:amd64.
Preparing to unpack .../libfluidsynth1_1.1.9-1_amd64.deb ...
Unpacking libfluidsynth1:amd64 (1.1.9-1) ...
Setting up fluid-soundfont-gm (3.1-5.1) ...
Setting up libfluidsynth1:amd64 (1.1.9-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.4) ...
/sbin/ldconfig.real: /usr/local/lib/python3.6/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

     |████████████████████████████████| 5.6MB 36.5MB/s 
     |████████████████████████████████| 61kB 9.8MB/s 
     |████████████████████████████████| 1.4MB 18.9MB/s 
     |████████████████████████████████| 71kB 12.1MB/s 
     |████████████████████████████████| 204kB 57.0MB/s 
     |██████████████████████████

/usr/local/lib/python3.6/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


🎉 Done!
2.1.3
2.4.1


---  
---  
# Retrain LSTM Magenta Network

1. Use this section to train a network and collect the .mag file which hold all the weights  

*NOTE: Each below cell has shared Flags, therefore until fixed, you must reset the Runtime to wipe the flags*  

---  
**IMPORT WORKING DIRECTORY AND CONVER TO NOTESEQUENCE**

In [ ]:
#import collab directory

from google.colab import files

---  
### **DO NOT RUN THE BELOW CELLS WITHOUT FIRST ASKING JAMES**

These cells are the training, but are connected to paths that we do not want to be overwritten

In [ ]:
#DO NOT RUN!!!#
#IMPORTS
from magenta.scripts import convert_dir_to_note_sequences

print("IMPORTS COMPLETE!\n")


#FUNCTION SETUP
#CREATES A "notesequences.tfrecord" file

!convert_dir_to_note_sequences \
 --input_dir="/content/drive/Shareddrives/Senior Design - Audio Project/MIDI Datasets/1001 Chord Progressions Dataset" \
 --output_file="/content/drive/Shareddrives/Senior Design - Audio Project/MIDI Datasets/1001 Chord Progressions Dataset/notesequences1001.tfrecord" \
 --recursive
 #DO NOT RUN#

IMPORTS COMPLETE!

2021-02-10 20:52:25.074683: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
/usr/local/lib/python3.6/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Converting files in '/content/drive/Shareddrives/Senior Design - Audio Project/MIDI Datasets/1001 Chord Progressions Dataset/'.
I0210 20:52:28.166122 139865395439488 convert_dir_to_note_sequences.py:83] Converting files in '/content/drive/Shareddrives/Senior Design - Audio Project/MIDI Datasets/1001 Chord Progressions Dataset/'.
INFO:tenso

In [ ]:
from magenta.models.polyphony_rnn import polyphony_rnn_create_dataset

#Create Sequence Examples from "notesequences.tfrecord"
!polyphony_rnn_create_dataset \
  --input="/content/drive/Shareddrives/Senior Design - Audio Project/MIDI Datasets/ChordProgressionTrainingFiles/ALLnotesequences.tfrecord" \
  --output_dir="/content/drive/Shareddrives/Senior Design - Audio Project/MIDI Datasets/ChordProgressionTrainingFiles/trainingsets" \
  --eval_ratio=0.10

In [ ]:
from magenta.models.polyphony_rnn import polyphony_rnn_train
print("TRAINING STARTED")
!polyphony_rnn_train \
  --run_dir="/content/drive/Shareddrives/Senior Design - Audio Project/MIDI Datasets/ChordProgressionTrainingFiles/CheckPoints" \
  --sequence_example_file="/content/drive/Shareddrives/Senior Design - Audio Project/MIDI Datasets/ChordProgressionTrainingFiles/trainingsets/training_poly_tracks.tfrecord" \
  --hparams="batch_size=64, rnn_layer_sizes=[128,128,128]" \
  --num_training_steps=1

In [ ]:
#Save the .mag file for later imports
!polyphony_rnn_generate \
  --run_dir="/content/drive/Shareddrives/Senior Design - Audio Project/MIDI Datasets/ChordProgressionTrainingFiles/CheckPoints" \
  --hparams="batch_size=128,rnn_layer_sizes=[128,128,128]" \
  --bundle_file="/content/drive/Shareddrives/Senior Design - Audio Project/MIDI Datasets/ChordProgressionTrainingFiles/Mag/polyphony_rnn.mag" \
  --save_generator_bundle

Initializing Polophony RNN...
INFO:tensorflow:Restoring parameters from /tmp/tmpagnzuhw7/model.ckpt
🎉 Done!


#### Training Notes:

1. Best Paramters = "Accuracy = 0.81087065, Global Step = 3784, Loss = 0.7766265, Perplexity = 2.1741257"  

---  
END OF TRAINING
---  

# Demo Chord Progession Model

This is where the trained model can be initialized and used for testing purposes

In [ ]:
#import .mag file in SequenceGenerator Interface
from magenta.models.polyphony_rnn import polyphony_sequence_generator
from magenta.models.shared import sequence_generator_bundle
from note_seq.protobuf import generator_pb2
from note_seq.protobuf import music_pb2

magFileLocation = "/content/drive/Shareddrives/Senior Design - Audio Project/MIDI Datasets/ChordProgressionTrainingFiles/Mag/polyphony_rnn.mag"
# Initialize the model.
print("Initializing Melody RNN...")
bundle = sequence_generator_bundle.read_bundle_file(magFileLocation)
generator_map = polyphony_sequence_generator.get_generator_map()
polyphony_rnn = generator_map['polyphony'](checkpoint=None, bundle=bundle)
polyphony_rnn.initialize()

print('🎉 Done!')

Initializing Melody RNN...
INFO:tensorflow:Restoring parameters from /tmp/tmplgwxlwq2/model.ckpt
🎉 Done!


In [ ]:
#grab file to work with
#setup a way to import data
from google.colab import files
uploaded = files.upload()

#1 MIDI FILE ONLY
dictionaryKey = list(uploaded.keys())[0]
#check to make sure the above is of type string
assert isinstance(dictionaryKey, str)

print("\nFILE UPLOAD SUCCESSFUL.\n File Name: ",dictionaryKey)


#IMPORTANT NOTES#
#1) uploaded is a dictionary so you can upload/import multiple files
#2) USE note_seq.midi_to_note_sequence(uploaded["dictionary location name printed in above cell"])


#convert midi file to "notesequence"
importNoteSequence = note_seq.midi_to_note_sequence(uploaded[dictionaryKey])

print("Conversion Complete!")

Saving D# I-vii-iii-ii-iii [gospel].mid to D# I-vii-iii-ii-iii [gospel] (1).mid

FILE UPLOAD SUCCESSFUL.
 File Name:  D# I-vii-iii-ii-iii [gospel].mid
Conversion Complete!


In [ ]:
import time
# CODE TO RUN MODEL ON CUSTOM MUSIC #

#CHANGE INPUT TO "twinkle_twinkle" for example code
#CHANGE input to "importNoteSequence" for personal import

clock_start = time.time()
#IMPORTANT NOTE --> primer length for model is double the input notes
input_sequence = importNoteSequence # change this to teapot if you want
num_steps = 260 # change this for shorter or longer sequences
temperature = 1.3 # the higher the temperature the more random the sequence.

# Set the start time to begin on the next step after the last note ends.
last_end_time = (max(n.end_time for n in input_sequence.notes)
                  if input_sequence.notes else 0)
qpm = input_sequence.tempos[0].qpm 
seconds_per_step = 60.0 / qpm / polyphony_rnn.steps_per_quarter
total_seconds = num_steps * seconds_per_step

generator_options = generator_pb2.GeneratorOptions()
generator_options.args['temperature'].float_value = temperature
generate_section = generator_options.generate_sections.add(
  start_time=last_end_time + seconds_per_step,
  end_time=total_seconds)

sequence = polyphony_rnn.generate(input_sequence, generator_options)

#speed check
print("model time: ",(time.time() - clock_start)/6)
#Ask the model to continue the sequence.
test_midi = note_seq.note_sequence_to_midi_file(sequence,"test1.mid")
#speed check
print("Model + Midi Conversion:",time.time() - clock_start)


#plot graph and music plugin code
note_seq.plot_sequence(sequence)
note_seq.play_sequence(sequence, synth=note_seq.fluidsynth)

INFO:tensorflow:Need to generate 197 more steps for this sequence, will try asking for 985 RNN steps
INFO:tensorflow:Beam search yields sequence with log-likelihood: -1128.573608 
INFO:tensorflow:Need to generate 35 more steps for this sequence, will try asking for 175 RNN steps
INFO:tensorflow:Beam search yields sequence with log-likelihood: -1292.835938 
INFO:tensorflow:Need to generate 10 more steps for this sequence, will try asking for 50 RNN steps
INFO:tensorflow:Beam search yields sequence with log-likelihood: -1371.555908 
INFO:tensorflow:Need to generate 1 more steps for this sequence, will try asking for 5 RNN steps
INFO:tensorflow:Beam search yields sequence with log-likelihood: -1384.041504 
model time:  1.057905673980713
Model + Midi Conversion: 6.3667497634887695


# Single Run Code

*use this segments of code to do the following.  
1. Load a MIDI File & Convert to NoteSequence
2. Snip A NoteSequence
3. Feed the Sequence to a pre-built Model, with the intialization weighting a. file.  
**Note: This can also be used for a retrained model if you change the weighting file**

---  
## Initialize Melody RNN

In [ ]:
print('Downloading model bundle. This will take less than a minute...')

#needs to be changed in hardware to .mag we create
note_seq.notebook_utils.download_bundle('basic_rnn.mag', '/content/')

# Import dependencies.
from magenta.models.melody_rnn import melody_rnn_sequence_generator
from magenta.models.shared import sequence_generator_bundle
from note_seq.protobuf import generator_pb2
from note_seq.protobuf import music_pb2

# Initialize the model.
print("Initializing Melody RNN...")
bundle = sequence_generator_bundle.read_bundle_file('/content/basic_rnn.mag')
generator_map = melody_rnn_sequence_generator.get_generator_map()
melody_rnn = generator_map['basic_rnn'](checkpoint=None, bundle=bundle)
melody_rnn.initialize()

print('🎉 Done!')

Initializing Melody RNN...
'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from /tmp/tmprmc0if89/model.ckpt
🎉 Done!


In [ ]:
#grab file to work with
#setup a way to import data
from google.colab import files
uploaded = files.upload()

#1 MIDI FILE ONLY
dictionaryKey = list(uploaded.keys())[0]
#check to make sure the above is of type string
assert isinstance(dictionaryKey, str)

print("\nFILE UPLOAD SUCCESSFUL.\n File Name: ",dictionaryKey)


#IMPORTANT NOTES#
#1) uploaded is a dictionary so you can upload/import multiple files
#2) USE note_seq.midi_to_note_sequence(uploaded["dictionary location name printed in above cell"])

Saving F# iii - V7 - IV - vii - iii - I [pop].mid to F# iii - V7 - IV - vii - iii - I [pop].mid

FILE UPLOAD SUCCESSFUL.
 File Name:  F# iii - V7 - IV - vii - iii - I [pop].mid


---  
---  
## ACCESS MIDI DATA  

**Select folder to draw MIDI Files from**

*IMPORTANT!!! PLEASE READ!!!*
FOR files.upload() to work you musy have 

In [ ]:
#setup a way to import data
from google.colab import files
uploaded = files.upload()

#1 MIDI FILE ONLY
dictionaryKey = list(uploaded.keys())[0]
#check to make sure the above is of type string
assert isinstance(dictionaryKey, str)

print("\nFILE UPLOAD SUCCESSFUL.\n File Name: ",dictionaryKey)


#IMPORTANT NOTES#
#1) uploaded is a dictionary so you can upload/import multiple files
#2) USE note_seq.midi_to_note_sequence(uploaded["dictionary location name printed in above cell"])

Saving NES_Mega_Man_Wily_Battle.mid to NES_Mega_Man_Wily_Battle (3).mid

FILE UPLOAD SUCCESSFUL.
 File Name:  NES_Mega_Man_Wily_Battle.mid


---  
**Convert Midi Files to "notesequence" & Listen to file that you uploaded**

In [ ]:
#convert midi file to "notesequence"
importNoteSequence = note_seq.midi_to_note_sequence(uploaded[dictionaryKey])

print("Conversion Complete!")

Conversion Complete!


In [ ]:
#play the file
def playNote_seq_InMIDIFormat_GoogleCollab(notesequence):
  """
  prints a midi file and a song sheet, and open a handle to listen. Only works in 
  """
  print("Number of Notes: {}\n".format(len(notesequence.notes)))

  #print temp infor
  for i in range(len(notesequence.tempos)):
    print("QPM: {}\n".format(notesequence.tempos[i].qpm))

  # This is a colab utility method that visualizes a NoteSequence.
  note_seq.plot_sequence(notesequence)

  # This is a colab utility method that plays a NoteSequence.
  note_seq.play_sequence(notesequence,synth=note_seq.fluidsynth)



#call the above function
playNote_seq_InMIDIFormat_GoogleCollab(importNoteSequence)

Number of Notes: 45

QPM: 120.0



---  
## Run Base Model

**Snips a NoteSequence from 0,max_notes**

In [ ]:
def snipNoteSequence(importNoteSequence,max_notes = 30,drumYes = False):
  """
  This function allows you to take just part of a song, starts at the beginning in current state. In future state you will be able to pick the middle

  INPUTS:
  importNoteSequence: note sequence taken from MIDI file, we are now extracting just the melody from it
  max_notes: the max amount of the notes the function will take into the music_pb2.NoteSequence()
  drumYes: removes all drums, so we have only the melody. Change if you do want the drums

  SIDE NOTE:
  self.notes.add(instrument="") will only extract a specific instrument
  """
  #initialize sequence
  songSnip = music_pb2.NoteSequence()

  #loop through imported NoteSequence and adds notes until counter is done. 
  count = 0
  for note in importNoteSequence.notes:
    #print(note)
    songSnip.notes.add(pitch = note.pitch,start_time = note.start_time,end_time=note.end_time, is_drum=drumYes ,instrument=0,velocity=note.velocity)
    count += 1
    if count >= max_notes:
      break

  songSnip.tempos.add(qpm=importNoteSequence.tempos[0].qpm)

  print("Amount of Notes:\n",len(songSnip.notes))


  # This is a colab utility method that visualizes a NoteSequence.
  note_seq.plot_sequence(songSnip)
  # This is a colab utility method that plays a NoteSequence.
  note_seq.play_sequence(songSnip,synth=note_seq.fluidsynth)

  return songSnip, len(songSnip.notes)

#calls the above function
songSnip,numOfNotes = snipNoteSequence(importNoteSequence,max_notes = 45,drumYes = False)

Amount of Notes:
 45


---  
**Run model after all clipping has been done**

In [ ]:
#used for speed purposes only
import time

def runModel(songSnip,num_steps = 150,temperature = 1.3):
  """
  songSnip: input of NoteSequence after filtering
  num_steps: change this for shorter or longer sequences
  temperature: the higher the temperature the more random the sequence.
  """
  #time collect
  clock_start = time.time()


  #IMPORTANT NOTE --> primer length for model is double the input notes
  input_sequence = songSnip # change this to teapot if you want

  # Set the start time to begin on the next step after the last note ends.
  last_end_time = (max(n.end_time for n in input_sequence.notes)
                    if input_sequence.notes else 0)
  qpm = input_sequence.tempos[0].qpm 
  seconds_per_step = 60.0 / qpm / melody_rnn.steps_per_quarter
  total_seconds = num_steps * seconds_per_step

  generator_options = generator_pb2.GeneratorOptions()
  generator_options.args['temperature'].float_value = temperature
  generate_section = generator_options.generate_sections.add(
    start_time=last_end_time + seconds_per_step,
    end_time=total_seconds)

  #generates the model, which is like training but weights are locked
  sequence = melody_rnn.generate(input_sequence, generator_options)

  ########################################################################
  #speed check
  print("model time: ",(time.time() - clock_start)/6)
  #Ask the model to continue the sequence.
  test_midi = note_seq.note_sequence_to_midi_file(sequence,"test1.mid")
  #speed check
  print("Model + Midi Conversion:",time.time() - clock_start)
  #########################################################################


  #plot graph and music plugin code
  note_seq.plot_sequence(sequence)
  note_seq.play_sequence(sequence, synth=note_seq.fluidsynth)

  return sequence
  #END OF FUNCTION


#num_step parameter intialization
buffer = 32
#two options for inputs to num_steps. Can be changed in later version to decide whats best
minimumStep = numOfNotes + buffer #minimum num_steps input
songDouble = 2 * numOfNotes #this is the maximum num_steps

#run function
sequence = runModel(songSnip,num_steps = songDouble,temperature = 1.3)

NameError: ignored

# MISCELLANEOUS CODE  

In [ ]:
#####CODE TO DOWNLOAD A MIDI FILE STARTING WITH A "NOTE_SEQUENCE"
########################################################################################
fileName = "MegaMan_1_14_2021.mid"
# Creates file from a sequence


#download original song: importNoteSequence
#download created song: sequence

note_seq.sequence_proto_to_midi_file(sequence, fileName)
files.download(fileName)
########################################################################################

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>